In [ ]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim library
from modsim import *
import pandas as pd

# set the random number generator
np.random.seed(9)

# If this cell runs successfully, it produces no output.

### Define the functions we have used in class:

In [298]:
def bike_to_campus(state):
    """Move one bike from dorm to campus.
    
    state: bikeshare State object
    """
    #Exit if dorm have no bikes
    if state.dorm == 0:
        state.dorm_unhappy += 1
     #   print(f'A customer at dorm could not find a bike at minute {state.clock}')
        return
    state.campus += 1
    state.dorm -= 1

In [299]:
def bike_to_dorm(state):
    """Move one bike from campus to dorm.
    
    state: bikeshare State object
    """
    #Exit if campus have no bikes
    if state.campus == 0:
        state.campus_unhappy += 1
      #  print(f'A customer at campus could not find a bike at minute {state.clock}')
        return
    state.dorm += 1
    state.campus -= 1

In [300]:
def step(state, p1, p2):
    """Simulate one minute of time.
    
    state: bikeshare State object
    p1: probability of an dorm -> campus customer arrival
    p2: probability of a campus -> dorm customer arrival
    """
    state.clock += 1
    if flip(p1):
        bike_to_campus(state)
    if flip(p2):
        bike_to_dorm(state)


In [312]:
def run_simulation(state, p1, p2, num_steps):
    """Simulate the given number of time steps.
    
    p1: probability of an dorm->campus customer arrival
    p2: probability of a campus->dorm customer arrival
    num_steps: number of time steps
    """
    for i in range(num_steps):
        step(state,p1,p2)
        results_campus[i] = state.campus
        results_dorm[i] = state.dorm
    
    return [state.campus_unhappy,state.dorm_unhappy]

In [355]:
""" Get the total number of unhappy customer in both station based on the change of total number of bikes & number of bikes in each station
data: database contain [total_bikes, campus_bikes, dorm_bikes, campus_unhappy_average, dorm_unhappy_average, total_unhappy_average]
num_steps: number of time steps
test_times: test times to find the average of total_unhappy -> hihger accuracy
df: data in Pandas type

"""
data = []
num_steps = 60 # We choose 60 time steps
test_times = 15 # We test 15 times and get the average of 15 test times

for total_bikes in range(8,30):
    # We test the number of total bikes from 8 to 30
    for campus_bikes in range(total_bikes):
        # We check all posible case: the number of bikes at campus from 0 to the total_bikes
        bikeshare = State(campus = campus_bikes, dorm = total_bikes - campus_bikes , campus_unhappy = 0, dorm_unhappy = 0, clock = 0)
        
        #We caculate the average after 15 test times
        campus_unhappy = 0
        dorm_unhappy = 0
        for i in range(test_times):
            campus_unhappy += run_simulation(bikeshare, 0.3,0.3,num_steps)[0]
            dorm_unhappy += run_simulation(bikeshare, 0.3,0.3,num_steps)[1]
        campus_unhappy_average = int(campus_unhappy/test_times)
        dorm_unhappy_average = int(dorm_unhappy/test_times)
        
        #After 15 test time, append average data of each case to database
        data.append([total_bikes, campus_bikes, total_bikes - campus_bikes, campus_unhappy_average,dorm_unhappy_average,  campus_unhappy_average + dorm_unhappy_average])
#convert array data to pandas -> visualize & easy to sort
df = pd.DataFrame(data, columns=['total_bikes', 'campus_bikes', 'dorm_bikes','campus_unhappy_average','dorm_unhappy_average','total_unhappy_average'])

,total_bikes,campus_bikes,dorm_bikes,campus_unhappy_average,dorm_unhappy_average,total_unhappy_average
0,8,0,8,19,28,47
1,8,1,7,21,43,64
2,8,2,6,13,39,52
3,8,3,5,12,45,57
4,8,4,4,28,24,52
...,...,...,...,...,...,...
402,29,24,5,0,15,15
403,29,25,4,0,12,12
404,29,26,3,0,5,5
405,29,27,2,16,0,16


In [356]:
""" Sort the mininum number of unhappy customer and total bikes to find the most optimized number of bikes
"""
#sort the total of unhappy customer first. Next sort the total of bikes
df.sort_values(by=['total_unhappy_average','total_bikes'])

,total_bikes,campus_bikes,dorm_bikes,campus_unhappy_average,dorm_unhappy_average,total_unhappy_average
216,22,13,9,0,0,0
340,27,17,10,0,0,0
391,29,13,16,0,0,0
234,23,9,14,1,0,1
309,26,12,14,1,0,1
...,...,...,...,...,...,...
26,10,9,1,22,33,55
3,8,3,5,12,45,57
68,14,5,9,45,14,59
1,8,1,7,21,43,64
